CUDA 버전 확인하기

- !nvidia-smi
- !nvcc --version

둘의 차이점은
nvidia-smi는 해당 장치에서 설치 가능한 가장 높은 버전을 보여주고,
nvcc --version은 현재 설치된 cuda 버전을 보여줌

출처 : https://stackoverflow.com/questions/9727688/how-to-get-the-cuda-version

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
# PyTorch 2.x 버전 설치
try:
    # 기본적으로 https://pytorch.kr/get-started/locally/ 에서
    # cuda 버전과 패키지매니저에 맞는 설치 명령어를 확인 가능
    # %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    import torch
    print(torch.__version__)
except:
    pass

In [ ]:
# pytorch에서 사용할 함수들 호출하기
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

----------------------------------------------------------

# CNN 으로 이미지 분석하기

## ****1. Mnist 데이터로 Convolution2D로 구현하기****

In [ ]:
train_dataset = datasets.MNIST('./data', train=True, download=True)

In [ ]:
train_dataset[10004][0].resize((280, 280))

In [ ]:
from matplotlib import pyplot as plt

plt.figure()
plt.imshow(train_dataset[10004][0])
plt.show()

## 2. 이미지를 학습시키기 위한 처리

1. ToTensor: 이미지를 학습이 가능하도록 숫자화한다. (Image, np,ndarray[W, H, C] -> tensor[C, H, W])
https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html

2. Normalize: CNN 성능을 높이기 위한 처리 (채널별로 각각 다른 평균, 분산을 적용 가능하다.)
https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html

In [ ]:
# 이미지를 학습이 가능하도록 숫자화
to_tensor = transforms.ToTensor()

In [ ]:
image_tensor = to_tensor(train_dataset[10004][0])

In [ ]:
image_tensor.shape

In [ ]:
# 단일 채널이므로 하나의 채널에만 평균 0.1307, 분산 0.3081로 조정한다.
normalizer = transforms.Normalize((0.1307,), (0.3081,))

In [ ]:
normalizer(image_tensor)

알아서 적용할수는 없을까?

-> 당연히 있다.

transforms.Compose([  
&nbsp;&nbsp;&nbsp;&nbsp;적용할 변형들  
])

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])

In [ ]:
# train_dataset에 transform 적용
train_dataset.transform = transform

## DataLoader 생성

- PyTorch는 모델을 학습시킬 때 batch_size만큼 데이터를 뽑아서 Dataset의 처리를 진행한 뒤 모델에 주입한다.
- 지금은 Dataset을 커스텀하여 사용하지 않고 있지만, 이러한 처리를 DataLoader가 해준다는 것은 분명 큰 이점이다.
- n_workers 옵션은 이를 병렬적으로 수행 가능하게 한다.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

## 모델 구축

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()

        # self.name
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, 1),
            nn.ReLU(True),
            nn.Flatten(),

            nn.Linear(2304, 32),
            nn.ReLU(True),
            nn.Linear(32, 10),
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
cnn = MyCNN()

In [ ]:
cnn

In [ ]:
# 모델 학습을 위한 옵티마이저와 스케줄러 정의
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
cnn.to(device)

# loss를 multiclass 분류에 적합한 loss인 cross entropy loss를 사용
criterion = nn.CrossEntropyLoss().cuda()

optimizer = optim.Adadelta(cnn.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [ ]:
test_dataset = datasets.MNIST('./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

In [ ]:
# 모델 학습 (tensorflow는 fit 하나로 아래의 과정을 처리한다.)
epochs = 10
dry_run = False # 1 배치만 훈련

for epoch in range(1, epochs+1):
    # 학습
    cnn.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = cnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

    # 테스트
    cnn.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = cnn(data)
            test_loss += criterion(output, target).detach().sum()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    scheduler.step()

### 모델의 구조를 이해해보자.  
output shape : 피처맵의 크기 * 채널  
param : weight의 개수

In [ ]:
%pip install torchinfo
from torchinfo import summary
summary(cnn.model)

# ouptut image의 크기는?
# 가중치의 개수는?

In [ ]:
(3*3*1+1)*64
(3*3*64+1)*128
(3*3*128+1)*256
3*3*256
(2304+1)*32
(32+1)*10

## ****2.cifar10 데이터로 Convolution2D로 구현해보기****

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

train_dataset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_dataset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
train_dataset[0][0].shape

In [ ]:
# 역정규화
denormalize = transforms.Compose([
     transforms.Normalize((0., 0., 0.), (1/0.5, 1/0.5, 1/0.5)),
     transforms.Normalize((-0.5, -0.5, -0.5), (1., 1., 1.)),
     ])

In [ ]:
i = 5
denormalize(train_dataset[i][0])

In [ ]:
plt.figure(figsize=(20,20))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(denormalize(train_dataset[i][0]).permute(1, 2, 0)) # C, W, H -> W, H, C
    plt.xlabel(classes[train_dataset[i][1]])
plt.show()

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()

        # self.name
        # 각 이미지의 크기는 어떻게 변할까?
        self.convolution = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, 1),
            nn.ReLU(),)
        # 30 30 64
        # 15 15 64
        # 13 13 128
        # 6 6 128
        # 4 4 256

        self.fully_connected = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.convolution(x)
        x = torch.flatten(x, 1)
        output = self.fully_connected(x)
        return output

In [ ]:
model = MyCNN()

In [ ]:
model

In [ ]:
from torchinfo import summary
summary(model)

In [ ]:
# 모델 학습을 위한 옵티마이저와 스케줄러 정의
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

criterion = nn.CrossEntropyLoss().cuda()

optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [ ]:
# 모델 학습 (tensorflow는 fit 하나로 아래의 과정을 처리한다.)
epochs = 10
dry_run = False # 1 배치만 훈련

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
            if dry_run:
                break

    # 테스트
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).detach().sum()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testloader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

    scheduler.step()

### CNN 응용모델들

https://pytorch.org/vision/stable/models.html

In [ ]:
import torchvision.models as tc_models
print(dir(tc_models))

In [ ]:
resnet = tc_models.resnet50(weights="IMAGENET1K_V2")

In [ ]:
resnet

In [ ]:
resnet = tc_models.resnet50(weights=None)

In [ ]:
import torch

# Option 1: passing weights param as string
model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")

# # Option 2: passing weights param as enum
# weights = torch.hub.load("pytorch/vision", "get_weight", weights="ResNet50_Weights.IMAGENET1K_V2")
# model = torch.hub.load("pytorch/vision", "resnet50", weights=weights)